<a href="https://colab.research.google.com/github/peterrrock2/gerrychain_migration_settings/blob/main/GerryChain_Migration_sln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

# version="old"
version="new"

if [ ! -f gerrymandria.json ]
then
  wget -q https://raw.githubusercontent.com/mggg/GerryChain/refs/heads/main/docs/_static/gerrymandria.json
  wget -q https://raw.githubusercontent.com/mggg/GerryChain/refs/heads/main/docs/_static/05_bg_census_consolidated.json -O BG_05.json
  wget -q https://github.com/mggg-states/PA-shapefiles/raw/refs/heads/master/PA.zip
  mkdir PA; unzip -qq PA.zip -d PA

  if [[ $version == "old" ]]
  then
    wget -q https://raw.githubusercontent.com/peterrrock2/gerrychain_migration_settings/refs/heads/main/old_gc/pyproject.toml
    wget -q https://raw.githubusercontent.com/peterrrock2/gerrychain_migration_settings/refs/heads/main/old_gc/uv.lock
  else
    wget -q https://raw.githubusercontent.com/peterrrock2/gerrychain_migration_settings/refs/heads/main/new_gc/pyproject.toml
    wget -q https://raw.githubusercontent.com/peterrrock2/gerrychain_migration_settings/refs/heads/main/new_gc/uv.lock

    echo "Please restart the runtime"
  fi

  # Point uv's "project environment" at the running Python's prefix
  export UV_PROJECT_ENVIRONMENT="$(python -c 'import sysconfig; print(sysconfig.get_config_var("prefix"))')"

  # Install exactly what's in uv.lock WITHOUT rewriting it
  # --inexact avoids removing Colab's preinstalled packages
  # --no-install-project installs deps but not local project as a package
  uv sync -q --frozen --inexact --no-dev --no-install-project
fi

#Task 1

Set up a basic ReCom chain on `gerrymandria.json` with

- A population and cut-edge updater
- Assigment using "district"
- Ideal population derived from the population updater
- Allowable popluation deviation of 1%
- 5000 steps

In [2]:
from gerrychain import Graph, Partition, MarkovChain, Election, GeographicPartition
from gerrychain.updaters import cut_edges, Tally
from gerrychain.accept import always_accept
from gerrychain.tree import bipartition_tree
from gerrychain.proposals import recom
from gerrychain.optimization import SingleMetricOptimizer


import pandas as pd
import numpy as np
from functools import partial


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [3]:
graph = Graph.from_json("./gerrymandria.json")

my_updaters = {
  "population": Tally("TOTPOP"),
  "cut_edges": cut_edges
}

initial_partition = Partition(
    graph,
    assignment="district",
    updaters=my_updaters
)

ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

proposal = partial(
    recom,
    pop_col="TOTPOP",
    pop_target=ideal_population,
    epsilon=0.01,
    node_repeats=2
)

recom_chain = MarkovChain(
    proposal=proposal,
    constraints=[],
    accept=always_accept,
    initial_state=initial_partition,
    total_steps=5000
)

In [4]:
assignment_list = []

for item in recom_chain.with_progress_bar():
    assignment_list.append(item.assignment)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


  0%|          | 0/5000 [00:00<?, ?it/s]

#Task 2

Set up a region-aware ReCom chain on `gerrymandria.json` with the same settings as before

- A population derived from the "TOTPOP" column and cut-edge updater
- Assigment using "district"
- Ideal population derived from the population updater
- Allowable popluation deviation of 1%
- 5000 steps

but with the following modifications:

- Surcharge "muni" by 0.5
- Surcharge "water_dist" by 0.5
- Change the bipartition_tree method to use 'random.choice' for selecting a cut edge.

In [5]:
graph = Graph.from_json("./gerrymandria.json")

my_updaters = {
    "population": Tally("TOTPOP"),
    "cut_edges": cut_edges
}

initial_partition = Partition(
    graph,
    assignment="district",
    updaters=my_updaters
)

ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

proposal = partial(
    recom,
    pop_col="TOTPOP",
    pop_target=ideal_population,
    epsilon=0.01,
    node_repeats=1,
    region_surcharge={
        "muni": 0.5,
        "water_dist": 0.5
    },
    # method = partial(
    bipartition_tree_fn = partial(
        bipartition_tree,
        max_attempts=100,
        allow_pair_reselection=True  # <-- This is the only change
    )
)

recom_chain = MarkovChain(
    proposal=proposal,
    constraints=[],
    accept=always_accept,
    initial_state=initial_partition,
    total_steps=5000,
)

In [6]:
assignment_list = []

for item in recom_chain.with_progress_bar():
    assignment_list.append(item.assignment)

  0%|          | 0/5000 [00:00<?, ?it/s]

#Task 3

Set up a region-aware ReCom chain on "PA_VTDs.json"

- A population updater derived from the "TOTPOP" column.
- Election updaters for the "PRES12" and "PRES16" elections
- Assignment using "538DEM"
- Ideal population derived from the population updater
- Allowable popluation deviation of 2%
- 200 steps

but with the following modifications:

Surcharge "muni" by 0.5
Surcharge "water_dist" by 0.5
Change the bipartition_tree method to use 'random.choice' for selecting a cut edge.

In [7]:
graph = Graph.from_file("./PA")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [8]:
elections = [
    Election("SEN10", {"Democratic": "SEN10D", "Republican": "SEN10R"}),
    Election("SEN12", {"Democratic": "USS12D", "Republican": "USS12R"}),
    Election("SEN16", {"Democratic": "T16SEND", "Republican": "T16SENR"}),
    Election("PRES12", {"Democratic": "PRES12D", "Republican": "PRES12R"}),
    Election("PRES16", {"Democratic": "T16PRESD", "Republican": "T16PRESR"})
]

my_updaters = {"population": Tally("TOTPOP", alias="population")}
election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)

initial_partition = GeographicPartition(
    graph,
    assignment="538DEM",
    updaters=my_updaters
)

ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

proposal = partial(
    recom,
    pop_col="TOTPOP",
    pop_target=ideal_population,
    epsilon=0.02,
    node_repeats=2
)

chain = MarkovChain(
    proposal=proposal,
    constraints=[],
    accept=always_accept,
    initial_state=initial_partition,
    total_steps=200
)


In [9]:
data = pd.DataFrame(
    sorted(partition["SEN12"].percents("Democratic"))
    for partition in chain.with_progress_bar()
)

  0%|          | 0/200 [00:00<?, ?it/s]

#Task 5

Set up a short-bursts SingleMetricOptimizer

- A population updater derived from the "tot_pop_20" column.
- Popluation updaters for the "VAP" and "BVAP" columns
- Assignment from a random assignment with 35 districts and population deviation of 2%
- Ideal population derived from the population updater
- Allowable popluation deviation of 2%
- 200 bursts of length 5
- Minimizes the number of cut edges


In [10]:
graph = Graph.from_json("BG_05.json")

POPCOL = "tot_pop_20"
SEN_DISTS = 35
EPS = 0.02
# TOTPOP = sum(graph.nodes()[n][POPCOL] for n in graph.nodes())
TOTPOP = sum(graph.node_data(n)[POPCOL] for n in graph.nodes)

chain_updaters = {
"population": Tally(POPCOL, alias="population"),
"VAP": Tally("VAP"),
"BVAP": Tally("BVAP")
}

initial_partition = Partition.from_random_assignment(
    graph=graph,
    n_parts=SEN_DISTS,
    epsilon=EPS,
    pop_col=POPCOL,
    updaters=chain_updaters
)

proposal = partial(
    recom,
    pop_col=POPCOL,
    pop_target=TOTPOP/SEN_DISTS,
    epsilon=EPS,
    node_repeats=1
)


def num_cut_edges(partition):
    return len(partition["cut_edges"])

optimizer = SingleMetricOptimizer(
    proposal=proposal,
    constraints=[],
    initial_state=initial_partition,
    optimization_metric=num_cut_edges,
    maximize=False
)


In [11]:
total_steps = 1000
burst_length = 5

min_scores_sb = np.zeros(total_steps)
for i, part in enumerate(optimizer.short_bursts(burst_length, total_steps//burst_length, with_progress_bar=True)):
    min_scores_sb[i] = optimizer.best_score

100%|██████████| 1000/1000 [00:14<00:00, 68.92it/s]
